This is the final version of the training workflow/pipeline

In [1]:
# import the necessary libraries
from ultralytics import YOLO
import os
import shutil
from megadetector.detection import run_detector
import cv2
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
import re
%matplotlib inline

In [2]:
def crop_image_normalized(img, normalized_coords):      
    try:
        
        
        x_min = bbox_normalized[0][0].int().item()
        y_min = bbox_normalized[0][1].int().item()
        x_max = bbox_normalized[0][2].int().item()
        y_max = bbox_normalized[0][3].int().item()
        
        cropped_img = image[y_min:y_max, x_min:x_max]
        return cropped_img

    except FileNotFoundError:
        print(f"Error: Image not found at {image_path}")
        return None
    except ValueError:
        print(f"Error: Invalid normalized coordinates format: {normalized_coords}. Expected 'x y w h'.")
        return None

In [ ]:
all_detections = []
image_source_directory = "E:\\Wild deserts photos\\2024_11_WCAM_originals\\WCAM16"
image_training_output = f"{image_source_directory}\\training"
image_paths = []
for dirpath, dirnames, filenames in os.walk(image_source_directory):
    for filename in filenames:
        if filename.endswith(".JPG"):
            image_path = os.path.join(dirpath, filename)
            image_paths.append(image_path)
for image_path in image_paths:
# try:
    print(f"Processing image: {image_path}")
    # get time of day from image
    image = cv2.imread(image_path)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hue = hsv_image[:, :, 0].mean()  # H channel represents hue
    # Adjust the thresholds as needed
    if hue > 10:
        model_required = 'day'
    else:
        model_required = 'night'

    print(f"Image is a {model_required} image")

    if model_required == 'night':
        detection_model = YOLO("E:\\Wild deserts photos\\model_training/night_no_fences_v2/19_05_20255/weights/best.pt")
        model = YOLO("E:\\Wild deserts photos\\model_training\\runs\\classify\\classificationv1_night\\weights\\best.pt")
    else:
        detection_model = YOLO("E:\\Wild deserts photos\\model_training/dayv4/19_05_20252/weights/best.pt")
        model = YOLO("E:\\Wild deserts photos\\model_training\\runs\\classify\\classificationv1_day\\weights\\best.pt")


    camera = re.search(r'WCAM(\d+)', image_path)[0]
    detections = detection_model.predict(image, save=False, # I don't save anything here because I want to sort into folders
            save_txt=False,
            save_conf=False, iou = 0.5, single_cls = True)
    datetime = Image.open(image_path)._getexif()[36867].replace(' ', '_').replace(':', '_')
    above = [d for d in detections[0].boxes if (d.conf > 0.7)]
    if not above:
        species = "empty"    
    for idx, detection in enumerate(above):
        bbox_normalized = detection.xyxy
        cropped_img = crop_image_normalized(image, bbox_normalized)
        

        result = model.predict(
            cropped_img, 
            save=False, # I don't save anything here because I want to sort into folders
            save_txt=False,
            save_conf=False)[0]
        species = result.cpu().names[result.probs.top1]
        conf = result.probs.cpu().top1conf.item()
        

        if conf<0.95:
            output_dir = f"{image_training_output}\\{model_required}\\low_confidence\\{species}"
        else:
            output_dir = f"{image_training_output}\\{model_required}\\{species}"
        print(f"Saving to {output_dir}")
        os.makedirs(output_dir, exist_ok=True)
        output_filename = f"{output_dir}\\{camera}_{datetime}_{species}_{idx}.jpg"
        cv2.imwrite(output_filename, cropped_img)
        df = pd.DataFrame({
                    'original_path': [image_path],
                    'cropped_path': [output_filename],
                    'species': [species],
                    'confidence': [conf],
                    'bbox': [bbox_normalized],
                    'time': [datetime],
                    'model': [model_required],
                    'camera': [camera] 
                })
        all_detections.append(df)
    output_dir_img = f"{image_training_output}\\detection\\{model_required}\\images"
    os.makedirs(output_dir_img, exist_ok=True)
    output_filename = f"{output_dir_img}\\{camera}_{datetime}.jpg"
    cv2.imwrite(output_filename, image)
    output_dir_txt = f"{image_training_output}\\detection\\{model_required}\\labels"
    output_filename_txt = f"{output_dir_txt}\\{camera}_{datetime}.txt"
    detections[0].save_txt(output_filename_txt)
# except Exception as e:
#     print(f"Error processing {image_path}: {e}")
if all_detections:
    final_df = pd.concat(all_detections, ignore_index=True)
    final_df.to_csv(f"{image_source_directory}\\output_{camera}.csv", index=False)



Processing image: E:\Wild deserts photos\2024_11_WCAM_originals\WCAM16\RCNX0020.JPG
Image is a night image

0: 480x640 1 item, 1791.4ms
Speed: 11.2ms preprocess, 1791.4ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 224x224 Kangaroo 0.95, Dingo 0.02, Cat 0.01, Rabbit 0.01, Fox 0.00, 214.3ms
Speed: 7.6ms preprocess, 214.3ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)
Saving to E:\Wild deserts photos\2024_11_WCAM_originals\WCAM16\training\night\Kangaroo
Processing image: E:\Wild deserts photos\2024_11_WCAM_originals\WCAM16\RCNX0013.JPG
Image is a night image

0: 480x640 1 item, 2488.7ms
Speed: 4.0ms preprocess, 2488.7ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 224x224 Rabbit 0.52, Cat 0.32, Fox 0.09, Dingo 0.05, Bilby 0.01, 309.8ms
Speed: 5.4ms preprocess, 309.8ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)
Saving to E:\Wild deserts photos\2024_11_WCAM_originals\WCAM16\training\night\low_confide

'c:\\Users\\willo\\OneDrive - UNSW\\Documents\\Work\\CES\\Wild Deserts\\Image classification\\coding'